Import the libraries required for the text generation

In [1]:
import tensorflow as tsf

import numpy as ny
import os
import time


Download the data

In [2]:
path_to_file = tsf.keras.utils.get_file('ebook','https://www.gutenberg.org/cache/epub/35993/pg35993.txt')

1335296/1321209 [==============================] - 0s 0us/step


Read the data

In [3]:
# Read, then decode for py2 compat.
content = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(content)))



Length of text: 1321107 characters


In [4]:
# Take a look at the first 250 characters in text
print(content[:250])

﻿The Project Gutenberg EBook of The History of Don Quixote de la Mancha, by 
Miguel de Cervantes Saavedra

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-u


In [5]:
# The unique characters in the file
voc = sorted(set(content))
print ('{} unique characters'.format(len(voc)))

94 unique characters




Vectorize the text

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(voc)}
idx2char = ny.array(voc)

text_asint = ny.array([char2idx[c] for c in content])

The prediction task


In [7]:
# The maximum length sentence we want for a single input in characters
seq_len = 100
epoch = len(content)//(seq_len+1)

# Create training examples / targets
char_dataset = tsf.data.Dataset.from_tensor_slices(text_asint)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
T
h
e
 


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [8]:
seq = char_dataset.batch(seq_len+1, drop_remainder=True)

for item in seq.take(5):
  print(repr(''.join(str(v) for v in idx2char[item.numpy()])))

'\ufeffThe Project Gutenberg EBook of The History of Don Quixote de la Mancha, by \r\nMiguel de Cervantes Saa'
'vedra\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions wha'
'tsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg Licens'
'e included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: The History of Don Quixote de '
'la Mancha\r\n\r\nAuthor: Miguel de Cervantes Saavedra\r\n\r\nRelease Date: April 29, 2011 [EBook #35993]\r\n\r\nL'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [9]:
def split_input_target(chunk):
  ip_txt = chunk[:-1]
  tg_txt = chunk[1:]
  return ip_txt, tg_txt

data_set = seq.map(split_input_target)

Print the input and target values:

In [10]:
for input, target in  data_set.take(1):
  print ('Input data: ', repr(''.join(map(str,idx2char[input.numpy()]))))
  print ('Target data:', repr(''.join(map(str,idx2char[target.numpy()]))))

Input data:  '\ufeffThe Project Gutenberg EBook of The History of Don Quixote de la Mancha, by \r\nMiguel de Cervantes Sa'
Target data: 'The Project Gutenberg EBook of The History of Don Quixote de la Mancha, by \r\nMiguel de Cervantes Saa'


Create training batches

In [11]:
# Batch size
BATCH_SIZE = 32

# Buffer size to shuffle the dataset
BUFFER_SIZE = 100

data_set = data_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

data_set

<BatchDataset element_spec=(TensorSpec(shape=(32, 100), dtype=tf.int64, name=None), TensorSpec(shape=(32, 100), dtype=tf.int64, name=None))>

In [12]:
# Length of the vocabulary in chars
voc_size = len(voc)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Build The Model

Use tsf.keras.Sequential to define the model.

In [13]:
def build_model(voc_size, embedding_dim, rnn_units, batch_size):
  model = tsf.keras.Sequential([
    tsf.keras.layers.Embedding(voc_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tsf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tsf.keras.layers.Dense(voc_size),
  ])
  return model

In [14]:

model = build_model(
    voc_size = len(voc),
    embedding_dim = embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [15]:
for input_batch, target_batch in data_set.take(1):
  batch_prediction = model(input_batch)
  print(batch_prediction.shape, "# (batch_size, sequence_length, vocab_size)")

(32, 100, 94) # (batch_size, sequence_length, vocab_size)


In [16]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (32, None, 256)           24064     
                                                                 
 lstm (LSTM)                 (32, None, 1024)          5246976   
                                                                 
 dense (Dense)               (32, None, 94)            96350     
                                                                 
Total params: 5,367,390
Trainable params: 5,367,390
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. 


In [17]:
sample_indices = tsf.random.categorical(batch_prediction[0], num_samples=1)
sample_indices = tsf.squeeze(sample_indices,axis=-1).numpy()
# a prediction of the next character index:
sample_indices

array([36, 51, 17,  4, 42, 52, 55,  8, 93, 45, 20, 25, 43, 10, 61, 85, 12,
       42, 20, 56,  0, 68,  1, 26, 87, 74, 90,  7, 59, 56,  1, 81, 36, 50,
       80, 86, 77,  8, 44, 19, 20, 34, 71, 72, 31, 22, 12, 28, 29, 26, 40,
       50, 20, 46, 29, 62, 35, 85, 19, 70, 61, 79, 44, 22, 69, 33, 49, 73,
       71, 56, 40, 79,  1, 32, 93, 35, 49, 14, 74, 30, 19, 76, 51, 60, 58,
       58, 25, 76, 88, 61, 74, 56, 51, 20, 45, 12, 16, 63,  5,  9])

Train the model


In [18]:
def loss(labels, logits):
  return tsf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

batch_loss  = loss(target_batch,batch_prediction)
print("Prediction shape: ", batch_prediction.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", batch_loss.numpy().mean())

Prediction shape:  (32, 100, 94)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.545524


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [19]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tsf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 20 epochs to train the model. 

In [ ]:

EPOCHS=20
history = model.fit(data_set, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
 29/408 [=>............................] - ETA: 36:06 - loss: 3.5377

Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [ ]:
tsf.train.latest_checkpoint(checkpoint_dir)
model = build_model(voc_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tsf.train.latest_checkpoint(checkpoint_dir))

model.build(tsf.TensorShape([1, None]))
model.summary()

The prediction loop


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 10000

  # Converting our start string to numbers (vectorizing)
  ip_eval = [char2idx[s] for s in start_string]
  ip_eval = tsf.expand_dims(ip_eval, 0)

  # Empty string to store our results
  text_generate = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temp = 2.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(ip_eval)
    # remove the batch dimension
    predictions = tsf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    prediction = predictions / temp
    predicted_id = tsf.random.categorical(prediction, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    ip_eval = tsf.expand_dims([predicted_id], 0)

    text_generate.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generate))

In [ ]:
print(generate_text(model, start_string=u"the"))